In [1]:
import findspark 
findspark.init('/home/bigdata/Documents/spark-3.0.0/')

In [2]:
from __future__ import print_function

import sys

In [3]:
from pyspark import SparkConf, SparkContext
from pyspark.streaming import StreamingContext
import os
from operator import add, sub
from pyspark.sql import Row, SparkSession
from pyspark.sql.functions import col

In [4]:
config = (SparkConf().setAppName("UNSW").set("spark.executor.memory", "2g"))

sc = SparkContext(conf=config)
ssc = StreamingContext(sc, 5)
ssc.checkpoint("checkpoint")

In [5]:
unsw_list = list()
# read the entire folder and add them into the Queue
entries = os.listdir('/home/bigdata/Downloads/UNSW_data')
for entry in entries:
    print(entry)
    path = "/home/bigdata/Downloads/UNSW_data/" + entry
    unsw_list += [sc.textFile(path, 5)]

UNSW-NB15-41.csv
UNSW-NB15-8.csv
UNSW-NB15-43.csv
UNSW-NB15-36.csv
UNSW-NB15-39.csv
UNSW-NB15-3.csv
UNSW-NB15-24.csv
UNSW-NB15-1.csv
UNSW-NB15-14.csv
UNSW-NB15-17.csv
UNSW-NB15-31.csv
UNSW-NB15-23.csv
UNSW-NB15-32.csv
UNSW-NB15-16.csv
UNSW-NB15-34.csv
UNSW-NB15-42.csv
UNSW-NB15-12.csv
UNSW-NB15-15.csv
UNSW-NB15-25.csv
UNSW-NB15-27.csv
UNSW-NB15-45.csv
UNSW-NB15-21.csv
UNSW-NB15-37.csv
UNSW-NB15-46.csv
UNSW-NB15-26.csv
UNSW-NB15-20.csv
UNSW-NB15-4.csv
UNSW-NB15-7.csv
UNSW-NB15-29.csv
UNSW-NB15-9.csv
UNSW-NB15-44.csv
UNSW-NB15-38.csv
UNSW-NB15-13.csv
UNSW-NB15-6.csv
UNSW-NB15-49.csv
UNSW-NB15-48.csv
UNSW-NB15-2.csv
UNSW-NB15-47.csv
UNSW-NB15-5.csv
UNSW-NB15-30.csv
UNSW-NB15-10.csv
UNSW-NB15-18.csv
UNSW-NB15-40.csv
UNSW-NB15-28.csv
UNSW-NB15-35.csv
UNSW-NB15-33.csv
UNSW-NB15-50.csv
UNSW-NB15-22.csv
UNSW-NB15-19.csv
UNSW-NB15-11.csv


In [6]:
unsw_list[49].take(2)

['"175.45.176.1","1043","149.171.126.18","53","udp","INT","3.0000001E-6","114","0","254","0","0","0","dns","1.52E+8","0","2","0","0","0","0","0","57","0","0","0","0","0","1424237372","1424237372","0.003","0","0","0","0","0","2","0","0","0","31","31","20","21","20","20","31","Generic","1"',
 '"175.45.176.1","1043","149.171.126.18","53","udp","INT","3.0000001E-6","114","0","254","0","0","0","dns","1.52E+8","0","2","0","0","0","0","0","57","0","0","0","0","0","1424237372","1424237372","0.003","0","0","0","0","0","2","0","0","0","31","31","20","21","20","20","31","Generic","1"']

In [7]:
rdd_stream = ssc.queueStream(unsw_list)

In [8]:
def AddNormalLabel(line):
            line =line.replace('"','').split(",")
            if line[47]== '':
                line[47]= 'Normal'
            elif line[47] == 'Backdoor':
                line[47] ='Backdoors'
            return line 

In [12]:
def getSparkSessionInstance(sparkConf):
    if ("sparkSessionSingletonInstance" not in globals()):
        globals()["sparkSessionSingletonInstance"] = SparkSession \
            .builder \
            .config(conf=sparkConf) \
            .getOrCreate()
    return globals()["sparkSessionSingletonInstance"]

Words = rdd_stream.map(AddNormalLabel)
  

In [13]:
Lmap= Words.filter(lambda x : x[47]) 
Attack_count = Lmap.map(lambda x : x[47])
Attackvalue = Attack_count.countByValue() 
Attackvalue.pprint() 

In [14]:
ssc.start()

-------------------------------------------
Time: 2020-09-22 12:50:05
-------------------------------------------
('Normal', 50512)
('DoS', 19)
('Fuzzers ', 61)
('Backdoors', 1)
('Reconnaissance ', 59)
('Worms', 2)
('Shellcode ', 14)
('Exploits', 111)
('Generic', 16)

-------------------------------------------
Time: 2020-09-22 12:50:10
-------------------------------------------
('Normal', 35674)
('Fuzzers ', 325)
('Backdoors', 12)
('DoS', 107)
('Reconnaissance ', 254)
('Shellcode ', 25)
('Worms', 3)
('Generic', 13828)
('Exploits', 537)
('Analysis', 30)

-------------------------------------------
Time: 2020-09-22 12:50:15
-------------------------------------------
('Normal', 42323)
('DoS', 584)
('Fuzzers ', 404)
('Backdoors', 10)
('Reconnaissance ', 298)
('Worms', 3)
('Shellcode ', 26)
('Generic', 5621)
('Exploits', 1441)
('Analysis', 85)



In [15]:
ssc.stop(stopSparkContext=True, stopGraceFully=True)

-------------------------------------------
Time: 2020-09-22 12:50:20
-------------------------------------------
('Normal', 50789)
('Reconnaissance ', 3)
('Exploits', 3)

-------------------------------------------
Time: 2020-09-22 12:50:25
-------------------------------------------
('Normal', 50795)

